# Introduction

Welcome to our code examples for the API! These examples show how to use different API endpoints to retrieve data, create calls and reserve time slots.

You can find the latest API documentation at: [ak API Documentation - Doctor Consultation ](https://arztkonsultation.de/ak-api-dokumentation/)


**Please note that you may need to customise the API URL.

## How do I use the Jupyter Notebook?

You can copy the ipynb file or run it directly in Google Colab.

You can then either

1. adapt the code so that you enter the e-mail address and password of the user who is to execute the API calls in the first commented-out section. You can then ignore the code in the second section ‘Authentication via config.ini’

or

2. create a config.ini file, enter the user's access data and save the file in Google Drive. You may need to change the path in the code


Please be aware that you should change the hostname to the URL we've provided, so that you can use the code examples properly.

## Good to know

You can find our API documentation on this page: https://arztkonsultation.de/ak-api-dokumentation/. We have multiple APIs. You probably mostly use the APIv2 (aka User API) and the Client API.

If you use the APIv2 you need to authenticate with a users email-adress and password to receive a Baerer token. Usually we recommend to use an general admin account / system account to communcate with our api.  

If you use the Client API you need a Client-ID and a password for authentication. If you don't have these data please concat us.


# Authentication and token retrieval



## APIv2

First, a user must authenticate themselves via email address and password (e.g. the admin user) in order to receive a token for subsequent API calls.

The following example sends a POST request for authentication and returns the access token.



In [ ]:
#Authentication via email and password


#email="Email address of the user who is to make the API call (e.g. doctor or admin)"
#password="Password of the account"

In [ ]:
# Authentication via config.ini
import os
from configparser import ConfigParser
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# The path to the config.ini file
config_file_path = "/content/drive/MyDrive/Colab Notebooks/config.ini"

# Ensure that the file exists
if not os.path.exists(config_file_path):
    raise FileNotFoundError(f"The file {config_file_path} was not found")

# Initialise ConfigParser and read the file
config = ConfigParser()
config.read(config_file_path)

# Read access data
email = config["credentials"]["email"]
password = config["credentials"]["password"]


Mounted at /content/drive


In [ ]:
# @title
import requests

#Change hostname!
hostname='app-sandbox.arztkonsultation.de'

def fetch(method, path, data=None, token=None):
    url = f'https://{hostname}{path}'
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    if token:
        headers['Authorization'] = f'Bearer {token}'

    response = requests.request(method, url, json=data, headers=headers)

    if response.status_code >= 400:
        raise Exception(f'Request failed with status code {response.status_code}: {response.text}')

    return response.json()

def login():
    c = {'email': email, 'password': password}
    result = fetch('POST', '/api/v2/auth', c)
    return result['access_token']

token = login()


## Client API



In [ ]:
# Authentication via client ID and password

client_id = 'your_client_id'  # Replace this with your client ID
pw = 'your_password'           # Replace this with your password

# @title
import requests

# Change Hostname!
hostname = 'app-sandbox.arztkonsultation.de'

def fetch(method, path, data=None, token=None):
    url = f'https://{hostname}{path}'
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    if token:
        headers['Authorization'] = f'Bearer {token}'

    response = requests.request(method, url, json=data, headers=headers)

    if response.status_code >= 400:
        raise Exception(f'Request failed with status code {response.status_code}: {response.text}')

    return response.json()

def login():
    c = {'client_id': client_id, 'pw': pw}
    result = fetch('POST', '/api/v2/client', c)
    return result['access_token']

token = login()
print("Access Token:", token)


# Users

Link to API-Doc: https://arztkonsultation.de/wp-content/uploads/2024/10/2024-10-01-ak-APIv2-Dokumentation.html#tag/Users

## Create user with the role ‘Doctor’

The script creates a user account with the role ‘Doctor’.


The user roles can be viewed here: https://arztkonsultation.de/wp-content/uploads/2024/05/2024-05-15-ak-APIv2-Dokumentation.html#section/Introduction/Roles

### Without notifications

A user with the role ‘Doctor’ is created and all e-mail & SMS notifications are deactivated.

In [ ]:
def create_user(token, name, email, pw):
    companies = fetch('GET', '/api/v2/companies', token=token)
    company_id = companies['data'][0]['id']
    user_data = {
        "name": name,
        "email": email,
        "password": pw,
        "company_id": company_id,
        "roles": [2],
        "notification_options": [
            {
              "notification_id": 1001, #Notificattion via E-Mail
              "enabled": False,
            },
            {
              "notification_id": 1002, #Notification via SMS
              "enabled": False,
            },
                ],

            }
    user = fetch('POST', '/api/v2/users', user_data, token=token)
    return user['data']


In [ ]:
user=create_user(token, "Arzt5 Code-Beispiele", "arzt5-ak-code-beispiele@arztkonsultation.de", "arzt5-ak-code-beispiele@arztkonsultation.de")
user

### With notifications

A user with the role ‘Doctor’ is created. The user receives notifications by e-mail when an appointment is created, changed or deleted. All other notifications have been deactivated in this example.

In [ ]:
def create_user(token, name, email, pw):
    companies = fetch('GET', '/api/v2/companies', token=token)
    company_id = companies['data'][0]['id']
    user_data = {
        "name": name,
        "email": email,
        "password": pw,
        "company_id": company_id,
        "roles": [2],
        "notification_options": [
            {
                "notification_id": 1001, #Notification by e-mail. Additional e-mail addresses can be stored to which a notification is to be sent, e.g. ‘value’: [‘test@em.ail’, ‘test2@andere.email’,],
                "enabled": True,
            },
            {
               "notification_id": 1002, #Notification by SMS. Additional telephone numbers can be stored to which a notification is to be sent, e.g. ‘value’: [‘+4912345679’,],
                "enabled": False,
            },
            {
                "notification_id": 2001, #Appointment created
                "enabled": True,
             },
             {
                "notification_id": 2002, #Appointment changed
                "enabled": True,
             },
             {
                "notification_id": 2003, ##Appointment deleted
                "enabled": True,
             },
             {
                "notification_id": 3001, #Document received
                "enabled": False,
             },
             {
                "notification_id": 3002, #Document sent
                "enabled": False,
             },
             {
                "notification_id": 3003, #Document deleted
                "enabled": False,
             },
             {
               "notification_id": 3004, #Document expires
                "enabled": False,
             },
             {
               "notification_id": 4001, #Invalid email of the patient
                "enabled": False,
             },
                ],

            }
    user = fetch('POST', '/api/v2/users', user_data, token=token)
    return user['data']

In [ ]:
user=create_user(token, "Arzt5 Code-Beispiele", "arzt5-ak-code-beispiele@arztkonsultation.de", "arzt5-ak-code-beispiele@arztkonsultation.de")
user

## List users

This example retrieves all users and displays the user ID, name and company ID.

Link to API-Doc:https://arztkonsultation.de/wp-content/uploads/2024/10/2024-10-01-ak-APIv2-Dokumentation.html#tag/Users/operation/User_index



In [ ]:
def getAllUsers(token):
    response = fetch('GET', '/api/v2/users?per_page=100', token=token)

    if 'data' in response and isinstance(response['data'], list):
        users = response['data']
    else:
        print("Error: Invalid API response")
        return []

    user_data = []
    for user in users:
        role_name = user['roles'][0]['name'] if 'roles' in user and len(user['roles']) > 0 else ''
        user_data.append([
            user.get('id', ''),
            user.get('title', ''),
            user.get('name', ''),
            user.get('company_id', ''),
            user.get('company_name', ''),
            user.get('ousubsidiary_name', ''),
            user.get('oulocation_name', ''),
            user.get('department_name', ''),
            role_name
        ])

    return user_data

# Call the function to retrieve the user information
users_data = getAllUsers(token)

# Table formatting with tabulate
headers = ['ID', 'Title', 'Display Name', 'Company', 'Company Name', 'Ousubsidiary Name', 'Oulocation Name', 'Department Name', 'Role']
print(tabulate(users_data, headers=headers, tablefmt='grid'))

# Calls

Link to API-Doc: https://arztkonsultation.de/wp-content/uploads/2024/10/2024-10-01-ak-APIv2-Dokumentation.html#tag/Calls

## Create Calls

In the following example, the admin account creates a video call with different scenarios and participants.

The ak users can join the video call either via the CallID and need then to authenticate via email/password, SSO or One-Time-Password (OTP).

**Default Mode is email/password**

*SSO needs to be activated and configured by ak. Please contact us, if you want to use it*

The code outputs the URLs for the stand-alone-video (SAV) for all participants. The SAV can be integrated via iFrame.

**Note**: The admin user will be added to the call as participant.


### Between one ak user & one TAN user (e.g. doctor to patient)

This code shows how to create a videocall between an ak user (e.g. doctor) and a TAN user (e.g. patient).

Depending on the role, the account that created the call can also be a participant (e.g. if the account has the role ‘doctor’)

In [ ]:
import requests
import json
from datetime import datetime


def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User12",
        "tan_email": "TAN-User2@arztkonsultation.de",
        "tan_data_agreed": True,
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288 #can be omitted if the account for the API call is the ak user who is also to join the call.
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']
    tan = next(user['tan'] for user in call['data']['users'] if user['role_name'] == 'tanuser')

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")

        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}/video/{call_id}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        else:
            if user['tan']:
                arzt_url = f"https://{hostname}/video/{call_id}?tan={user['tan']}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")
            else:
                arzt_url = f"https://{hostname}/video/{call_id}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")

    return {'callId': call_id, 'tan': tan}

# Call the create_call function with the received token
result = create_call(token)
print(result)


Call
{
  "id": 45982,
  "created_at": 1729003817,
  "updated_at": 1729003817,
  "deleted_at": null,
  "settings": {
    "queue_type": "",
    "offline": [],
    "timings": {
      "lead_time_create": 0,
      "lead_time_cancel": 0
    }
  },
  "parent": null,
  "children": null,
  "begin": null,
  "duration": 15,
  "room": "EtWElFniei4kGXT8U5LCTwmKMnmdVI",
  "status": 0,
  "extid": null,
  "description": null,
  "extendable": null,
  "source": null,
  "users": [
    {
      "id": 129907,
      "user_name": "TAN-User12",
      "status": 19,
      "gender": "notset",
      "company_id": null,
      "company_name": null,
      "ousubsidiary_id": null,
      "oulocation_id": null,
      "department_id": null,
      "department_name": null,
      "role_id": 1,
      "role_name": "tanuser",
      "tan": "KBTv-YhyT-vA3d",
      "tan_valid_until": 1737331200,
      "log_id": null,
      "email": "tan-user2@arztkonsultation.de",
      "mobile": null,
      "consentstatus": {
        "tan_descri

### Between *one* ak user & *two* TAN users (e.g. doctor to patient)

This code shows how to create a videocall between an ak user (e.g. doctor) and two TAN users (e.g. patient and relative).

Depending on the role, the account that created the call can also be the participant (e.g. if the account has the role ‘doctor’)

In [ ]:
import requests
import json
from datetime import datetime


def create_call(token):
    call_data = {
        "duration": 15,
        "begin": 1727884800,
        "tan_username": "TAN-User1",
        "tan_email": "TAN-User1@arztkonsultation.de",
        "tan_data_agreed": True,
        "call_type": "tan",
        "additional_callees": [
            {
                "name": "TAN2",
                "user_id": 59288 #can be omitted if the account for the API call is the ak user who is also to join the call.

            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']
    tan = next(user['tan'] for user in call['data']['users'] if user['role_name'] == 'tanuser')

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")

        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}.arztkonsultation.de/video/{call_id}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        else:
            if user['tan']:
                arzt_url = f"https://{hostname}.arztkonsultation.de/video/{call_id}?tan={user['tan']}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")
            else:
                arzt_url = f"https://{hostname}.arztkonsultation.de/video/{call_id}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")

    return {'callId': call_id, 'tan': tan}

# Call the create_call function with the received token
result = create_call(token)
print(result)


### Between *two* ak users and *one* TAN user
In the following example, the admin account creates a video call with one TAN user (‘TAN user1’) and two ak users.

In [ ]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "begin": 1727884800,
        "tan_username": "TAN-User1",
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288 #If necessary, only a user ID needs to be added if the account for the API call is the ak user who is also to join the call.
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
            display(IFrame(patient_url, width=800, height=600))
        elif user['role_name'] == 'callee':
            arzt_url = f"https://{hostname}/video/{call['data']['id']}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")


# Call the create_call function with the received token
create_call(token)


### Between *two* ak users (e.g. doctor consultation)
In the following example, the admin account creates a video call between two ak users.

***However, a TAN user must be added so that the ak users are assigned to the call and can join the call.***

The code outputs the URLs for all participants.


In [ ]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "begin": 1727884800,
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        elif user['role_name'] == 'callee':
            arzt_url = f"https://{hostname}/video/{call['data']['id']}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")


# Call the create_call function with the received token
create_call(token)


## Auth via One-Time-Password (OTP)

This example creates a call and also creates a OTP for the ak user. The OTP is valid fdor only 60s and needs therefore be created everytime before the user wants to join the call. The OTP is added via a query paramter in the URL.

The code performs the following tasks:

- Authenticates the user using email and password to obtain a Bearer token.
- Creates a video call with specified participants and duration.
- Generates an OTP (One-Time Password) for a specific user after the call is created.
- Formats and displays participant information and links to join the video call, including a link for the user with OTP.

In [ ]:
import os
import requests
import json
import base64
from datetime import datetime
from configparser import ConfigParser
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# The path to the config.ini file
config_file_path = "/content/drive/MyDrive/Colab Notebooks/config.ini"

# Ensure that the file exists
if not os.path.exists(config_file_path):
    raise FileNotFoundError(f"The file {config_file_path} was not found")

# Initialise ConfigParser and read the file
config = ConfigParser()
config.read(config_file_path)

# Print available sections for debugging
print("Available sections:", config.sections())

# Read access data without printing credentials
email = config["credentials"]["email"]
password = config["credentials"]["password"]
client_id = config["client_credentials"]["client_id"]
client_pw = config["client_credentials"]["client_password"]

# Specific email for OTP generation
otp_email = "arzt1-ak-code-beispiele@arztkonsultation.de"

def authenticate(email, password):
    url = "https://app-sandbox.arztkonsultation.de/api/v2/auth"
    data = {
        "email": email,
        "password": password
    }

    response = requests.post(url, json=data)
    try:
        response.raise_for_status()  # Raises an error for bad responses
        return response.json()['access_token']
    except KeyError:
        print("Error: 'access_token' key not found in the response.")
        print("Response content:", response.json())  # Print the full response for debugging
        raise  # Reraise the exception for further handling

def client_authenticate(client_id, client_pw):  # Updated variable name
    url = "https://app-sandbox.arztkonsultation.de/api/v2/client"
    data = {
        "client_id": client_id,
        "pw": client_pw
    }

    response = requests.post(url, json=data)
    try:
        response.raise_for_status()  # Raises an error for bad responses
        return response.json()['token']  # Assuming the token is in this field
    except KeyError:
        print("Error: 'token' key not found in the response.")
        print("Response content:", response.json())  # Print the full response for debugging
        raise  # Reraise the exception for further handling
    except requests.HTTPError as e:
        print("HTTP Error:", e.response.status_code)
        print("Response content:", e.response.json())  # Print the full response for debugging
        raise  # Reraise the exception for further handling

def fetch(method, endpoint, data, token):
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    url = f"https://app-sandbox.arztkonsultation.de{endpoint}"
    response = requests.request(method, url, headers=headers, json=data)
    response.raise_for_status()  # Raises an error for bad responses
    return response.json()

def create_call(token):
    call_data = {
        "duration": 15,
        "begin": 1727884800,
        "tan_username": "TAN-User1",
        "tan_email": "TAN-User1@arztkonsultation.de",
        "tan_data_agreed": True,
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288  # can be omitted if the account for the API call is the ak user who is also to join the call.
            }
        ]
    }

    # Create the call
    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']
    tan = next(user['tan'] for user in call['data']['users'] if user['role_name'] == 'tanuser')

    print('Call created successfully.')

    participants = call['data']['users']
    print("\nTeilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")

        if user['role_name'] == 'tanuser':
            patient_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tan={tan}"
            print(f"    - Link direkt ins Video für {user['user_name']}: {patient_url}")
        elif user['role_name'] == 'callee':
            # Store participant details for OTP generation
            callee_user = user

    # Authenticate to the client API for OTP generation
    scoped_token_data = {
        "email": otp_email,  # Use the specific user's email here
        "scope_type": "call",
        "scope_id": call_id  # Use the created call ID here
    }

    client_auth_token = client_authenticate(client_id, client_pw)  # Use Client ID and pw
    otp_response = fetch('POST', '/api/ext/createScopedToken', scoped_token_data, client_auth_token)  # Updated to correct endpoint

    # Debugging: Print the complete response from the OTP API
    print("OTP API Response:", otp_response)

    # Check the structure of the response for the token
    if 'token' in otp_response:  # Adjusted to check for the token directly
        otp = otp_response['token']  # Assuming the token is returned in this field
    else:
        print("Error: Expected 'token' key not found in the response.")
        raise ValueError("Invalid response structure")

    # Base64 encode the OTP
    base64_otp = base64.b64encode(otp.encode()).decode()

    # Construct the URL with the base64 encoded OTP
    video_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tid={base64_otp}"
    print(f"  - Link mit OTP direkt ins Video für Arzt1 Code-Beispiele: {video_url}")

# Authenticate to get the Bearer token
bearer_token = authenticate(email, password)

# Call the create_call function with the received Bearer token
create_call(bearer_token)


## List Calls

In [ ]:
import pandas as pd
from datetime import datetime, timezone

# Function for retrieving all past calls with dynamic pagination
def getPastCalls(token):
    page = 1
    all_calls = []
    per_page = 500

    current_time = datetime.now(timezone.utc).timestamp()  # Current time in UTC timestamp
    total_pages = None  # Initialize total_pages to None

    while True:
        # API call to fetch calls with pagination
        calls_response = fetch('GET', f'/api/v2/calls?per_page={per_page}&page={page}', token=token)

        if calls_response is None or 'data' not in calls_response:
            print("Error: No valid API response received")
            break

        calls = calls_response.get('data', [])

        # Process each call entry
        for call in calls:
            # Filter only calls that have a "begin" timestamp in the past
            if call.get('begin') and int(call['begin']) < current_time:
                created_at = datetime.utcfromtimestamp(int(call['created_at'])).strftime('%Y-%m-%d %H:%M:%S')
                updated_at = datetime.utcfromtimestamp(int(call['updated_at'])).strftime('%Y-%m-%d %H:%M:%S')
                begin = datetime.utcfromtimestamp(int(call['begin'])).strftime('%Y-%m-%d %H:%M:%S') if call.get('begin') else None

                # Handle None duration values by setting them to 0
                duration = int(call.get('duration', 0) or 0)

                # Process each user in the call (loop through all users)
                for user_info in call.get('users', []):
                      all_calls.append({
                          'call_id': call['id'],
                          'created_at': created_at,
                          'updated_at': updated_at,
                          'user_name': user_info.get('user_name', ''),
                          'begin': begin,
                          'duration': duration,
                          'status': call.get('status', ''),
                          'company_id': user_info.get('company_id', ''),
                          'tan': user_info.get('tan', ''),
                          'oulocation_id': user_info.get('oulocation_id', ''),
                          'department_name': user_info.get('department_name', ''),
                          'user_id': user_info.get('id', '')
                      })


        # Fetch the total number of pages from the API response metadata
        if total_pages is None and 'meta' in calls_response:
            total_pages = calls_response['meta'].get('last_page', 1)

        # Check if we've processed all pages
        if page >= total_pages:
            break

        # Go to the next page
        print(f"Fetched page {page}/{total_pages}")
        page += 1

    return all_calls

# Main function to retrieve past calls and display them in a table
def main():
    token = login()
    if not token:
        print("Error: No valid token received")
        return

    # Get all past calls for all users
    past_calls = getPastCalls(token)

    # Load calls into a DataFrame
    df_calls = pd.DataFrame(past_calls)

    # Display the DataFrame as a table
    from IPython.display import display
    display(df_calls)

# Execute the main function
main()


## Delete Call

A call can only be deleted if it has not yet been executed

In [ ]:
def delete_call(call_id, token):
    path = f'/api/v2/calls/{call_id}'
    url = f'https://{hostname}{path}'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"Der Anruf mit der ID {call_id} wurde erfolgreich gelöscht.")
    else:
        print(f"Fehler beim Löschen des Anrufs: {response.status_code}")
        print(response.text)
        raise Exception(f'Fehler beim Löschen des Anrufs: {response.status_code} {response.text}')

# Call the delete_call function with a call ID and token
call_id = '23666'  # Replace this with the desired call ID
delete_call(call_id, token)

# Calllogs

Link to API-Doc: https://arztkonsultation.de/wp-content/uploads/2024/10/2024-10-01-ak-APIv2-Dokumentation.html#tag/Calllogs

## Retrieve call logs

This example retrieves the call logs exclusively via the API endpoint ‘/api/v2/calllogs’. The users are retrieved in advance via ‘/api/v2/users’.


In [ ]:
def get_call_logs(token):
    # Retrieve call logs
    return fetch('GET', '/api/v2/calllogs', token=token)

# Main execution
try:
    token = login()
    call_logs = get_call_logs(token)
    print(call_logs)
except Exception as e:
    print(f'An error occurred: {e}')

## Extended call logs

This example retrieves the call logs via the API endpoint ‘/api/v2/calllogs’. In addition, further information about the users and calls is retrieved via ‘/api/v2/users’ and ‘/api/v2/calls’ and collated in a table.

In [ ]:
import pandas as pd
from datetime import datetime, timezone

# Function for retrieving all past calls with dynamic pagination
def getPastCalls(token):
    page = 1
    all_calls = []
    per_page = 500

    current_time = datetime.now(timezone.utc).timestamp()  # Current time in UTC timestamp
    total_pages = None  # Initialize total_pages to None

    while True:
        # API call to fetch calls with pagination
        calls_response = fetch('GET', f'/api/v2/calls?per_page={per_page}&page={page}', token=token)

        if calls_response is None or 'data' not in calls_response:
            print("Error: No valid API response received")
            break

        calls = calls_response.get('data', [])

        # Process each call entry
        for call in calls:
            # Filter only calls that have a "begin" timestamp in the past
            if call.get('begin') and int(call['begin']) < current_time:
                created_at = datetime.utcfromtimestamp(int(call['created_at'])).strftime('%Y-%m-%d %H:%M:%S')
                updated_at = datetime.utcfromtimestamp(int(call['updated_at'])).strftime('%Y-%m-%d %H:%M:%S')
                begin = datetime.utcfromtimestamp(int(call['begin'])).strftime('%Y-%m-%d %H:%M:%S') if call.get('begin') else None

                # Handle None duration values by setting them to 0
                duration = int(call.get('duration', 0) or 0)

                # Fetch the 'end' time for this call from the /calllogs endpoint
                calllog_response = fetch('GET', f'/api/v2/calllogs?call_id={call["id"]}', token=token)

                # Check if calllog_response is valid and has data
                if calllog_response and 'data' in calllog_response and calllog_response['data']:
                    calllog = calllog_response['data'][0]  # Get the first entry
                    end = datetime.utcfromtimestamp(int(calllog.get('end', 0))).strftime('%Y-%m-%d %H:%M:%S') if calllog.get('end') else None
                else:
                    end = None  # No call log entry found

                # Process each user in the call (loop through all users)
                for user_info in call.get('users', []):
                    all_calls.append({
                        'call_id': call['id'],
                        'created_at': created_at,
                        'updated_at': updated_at,
                        'user_name': user_info.get('user_name', ''),
                        'begin': begin,
                        'end': end,
                        'duration': duration,
                        'status': call.get('status', ''),
                        'company_id': user_info.get('company_id', ''),
                        'tan': user_info.get('tan', ''),
                        'oulocation_id': user_info.get('oulocation_id', ''),
                        'department_name': user_info.get('department_name', ''),
                        'user_id': user_info.get('id', '')
                    })

        # Fetch the total number of pages from the API response metadata
        if total_pages is None and 'meta' in calls_response:
            total_pages = calls_response['meta'].get('last_page', 1)

        # Check if we've processed all pages
        if page >= total_pages:
            break

        # Go to the next page
        print(f"Fetched page {page}/{total_pages}")
        page += 1

    return all_calls

# Main function to retrieve past calls and display them in a table
def main():
    token = login()
    if not token:
        print("Error: No valid token received")
        return

    # Get all past calls for all users
    past_calls = getPastCalls(token)

    # Load calls into a DataFrame
    df_calls = pd.DataFrame(past_calls)

    # Display the DataFrame as a table
    from IPython.display import display
    display(df_calls)

# Execute the main function
main()


# iFrame

This example shows how the video can be integrated via iFrame.

You can find integration instructions at the following link: https://arztkonsultation.de/wp-content/uploads/2024/05/2024-05-15-ak-iFrame-Integrationsanleitung.pdf.pdf

## Docs via CallID


In [ ]:
from IPython.display import IFrame

url = "https://app-sandbox.arztkonsultation.de/video/24761
IFrame(url, width=800, height=600)

## TAN user without waiting area

In [ ]:
from IPython.display import IFrame

url = "https://app-sandbox.arztkonsultation.de/video/24761?tan=FtUD-tu5e-ZsBA"
IFrame(url, width=800, height=600)


## TAN user with waiting area

In [ ]:
from IPython.display import IFrame

url = "https://app-sandbox.arztkonsultation.de/video/login?tan=FtUD-tu5e-ZsBA"
IFrame(url, width=800, height=600)